In [106]:
from datasets import load_dataset, ClassLabel
import datasets

In [107]:
raw_dataset = load_dataset("financial_phrasebank", "sentences_50agree")
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4846
    })
})

In [108]:
raw_dataset['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'neutral', 'positive'], id=None)}

In [109]:
label_mapping = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

df = raw_dataset['train'].to_pandas()
df['label'] = df['label'].apply(lambda x: label_mapping[x])
dataset = datasets.Dataset.from_pandas(df)
dataset[0]

{'sentence': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 'label': 'neutral'}

In [110]:
SYSTEM_PROMPT = """What is the sentiment of the below sentence? Please choose an answer from {{negative/neutral/positive}}.\n Sentence: 
"""

def add_system_prompt(example):
    example['sentence'] = SYSTEM_PROMPT + example['sentence']
    return example 

In [111]:
dataset = dataset.map(add_system_prompt)
dataset[0]

Map:   0%|          | 0/4846 [00:00<?, ? examples/s]

{'sentence': 'What is the sentiment of the below sentence? Please choose an answer from {{negative/neutral/positive}}.\n Sentence: \nAccording to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 'label': 'neutral'}

In [112]:
def create_message(example):
    example["message"] = [
        {"role": "user", "content": example['sentence']},
        {"role": "assistant", "content": example['label']}
    ]
    return example 

In [113]:
dataset = dataset.map(create_message)
dataset[0]

Map:   0%|          | 0/4846 [00:00<?, ? examples/s]

{'sentence': 'What is the sentiment of the below sentence? Please choose an answer from {{negative/neutral/positive}}.\n Sentence: \nAccording to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 'label': 'neutral',
 'message': [{'content': 'What is the sentiment of the below sentence? Please choose an answer from {{negative/neutral/positive}}.\n Sentence: \nAccording to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
   'role': 'user'},
  {'content': 'neutral', 'role': 'assistant'}]}

In [114]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

def apply_chat_template(example):
    example["text"] = tokenizer.apply_chat_template(conversation=example["message"], tokenize=False)
    return example


In [115]:
dataset = dataset.map(apply_chat_template)
dataset[0]

Map:   0%|          | 0/4846 [00:00<?, ? examples/s]

{'sentence': 'What is the sentiment of the below sentence? Please choose an answer from {{negative/neutral/positive}}.\n Sentence: \nAccording to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 'label': 'neutral',
 'message': [{'content': 'What is the sentiment of the below sentence? Please choose an answer from {{negative/neutral/positive}}.\n Sentence: \nAccording to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
   'role': 'user'},
  {'content': 'neutral', 'role': 'assistant'}],
 'text': '<s>[INST] What is the sentiment of the below sentence? Please choose an answer from {{negative/neutral/positive}}.\n Sentence: \nAccording to Gran , the company has no plans to move all production to Russia , although that is where the company is growing . [/INST]neutral</s>'}

In [116]:
dataset = dataset.remove_columns(["sentence", "label", "message"])

In [117]:
dataset.to_json("../../data/fpb_processed.jsonl")


Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

1418456